In [1]:
from dotenv import load_dotenv
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

load_dotenv('.env')

True

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

llm = ChatOllama(model='llama3.2:latest', base_url='http://localhost:11434')

## Define Tools

In [4]:
from langchain_community.tools import TavilySearchResults

@tool
def search(query: str) -> str:
    """
    Busca en la web información general actualizada y en tiempo real.
    Por ejemplo: noticias, actualizaciones del clima, últimas tendencias en IA, etc.

    Args:
        query: la consulta de búsqueda.
    """
    search = TavilySearchResults(
        max_results=5,
        search_depth='advanced',
        include_answer=True,
        include_raw_content=True,
    )
    response = search.invoke(query)
    return response
    

In [5]:
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

db_name = r'C:\Users\Jorge Moraga Calvo\Desktop\Python\langchain\operations_research'
vector_store = FAISS.load_local(db_name, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type='similarity',
                                      search_kwargs={'k': 3})

In [6]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBFE28AE70>, search_kwargs={'k': 3})

In [7]:
@tool
def operations_research(query: str) -> str:
    """
    Busca información sobre la historia y características de la investigación de operaciones.
    Para cualquier duda o pregunta sobre este aspecto, debes usar esta herramienta.
    Si la pregunta no está relacionada con la investigación de operaciones, simplementa response: `No sé sobre
    ese tema, está fuera de mi alcance`.

    Args:
        query: la consulta de búsqueda
    """

    response = retriever.invoke(query)
    return response

In [8]:
operations_research.invoke("¿Cómo es la formulación de un modelo matemático?")

[Document(id='efe628e5-c91f-469f-a334-7c4281b6637f', metadata={'producer': 'Adobe Acrobat 8.1', 'creator': 'Adobe Acrobat 8.1 Combine Files', 'creationdate': '2010-05-24T10:07:08-05:00', 'source': 'document_store/pdfs\\hillier_lieberman_proof.pdf', 'file_path': 'document_store/pdfs\\hillier_lieberman_proof.pdf', 'total_pages': 20, 'format': 'PDF 1.6', 'title': 'Introduccion a la investigacion de operaciones', 'author': 'Frederick S. Hillier', 'subject': '', 'keywords': '', 'moddate': '2010-07-07T13:08:58-05:00', 'trapped': '', 'page': 8}, page_content='■2.2 FORMULACIÓN DE UN MODELO MATEMÁTICO\nUna vez que el tomador de decisiones defi ne el problema, la siguiente etapa consiste en reformular-\nlo de manera conveniente para su análisis. La forma convencional en que la investigación de ope-\nraciones logra este objetivo es mediante la construcción de un modelo matemático que represente \nla esencia del problema. Antes de analizar cómo se elaboran los modelos de este tipo se explorará'),


### Agent
- The **agent** is responsible for taking in input and deciding what actions to take.
- The **agent** doesn't execute those actions - that is done by the `AgentExecutor`
- `create_tool_calling_agent` will call `.bind_tools` for us under the hood 

In [9]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [10]:
prompt = hub.pull('hwchase17/openai-functions-agent')

In [11]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
tools = [search, operations_research]
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
question = "¿Cuál son las últimas tendencias en IA?"
response = agent_executor.invoke({'input': question})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'Ñcutas óra de IA'}`


[{'url': 'https://creati.ai/pt/ai-tools/ora-ai/', 'content': 'A ORA é uma plataforma inovadora de IA que permite aos usuários criar chatbots altamente personalizáveis de forma rápida e eficaz. Com foco na facilidade de uso, a ORA elimina as complexidades geralmente associadas ao desenvolvimento de IA, tornando-se acessível tanto para novatos quanto para profissionais.'}, {'url': 'https://creati.ai/es/ai-tools/ora-ai/', 'content': 'ORA es una plataforma de IA innovadora que permite a los usuarios crear chatbots altamente personalizables de manera rápida y eficiente. Con un enfoque en la facilidad de uso, ORA elimina las complejidades asociadas a menudo con el desarrollo de IA, haciéndola accesible tanto para principiantes como para profesionales.'}, {'url': 'https://www.talkoraia.com.mx/', 'content': 'ORA IA La Primera Inteligencia Artificial Conversacional Hispanohablante. Participación I